# Tutorial 1: Simple Medical Image Segmentation on SageMaker

This notebook demonstrates how to train a medical image segmentation model on Amazon SageMaker using a single GPU.

## What You'll Learn
- Setting up a SageMaker training job
- Training a SegResNet model for 3D medical image segmentation
- Deploying the model to a SageMaker endpoint
- Running inference on medical images
- Cleaning up resources

## Prerequisites
- Medical imaging data in S3 (organized as train/valid/test folders)
- SageMaker execution role with S3 access

## Step 1: Setup and Imports

In [ ]:
import os
import sagemaker
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker import get_execution_role
import boto3

sagemaker_session = sagemaker.Session(boto3.Session(region_name='us-east-1'))
role = get_execution_role()
# IF the role is giving an error, uncomment the following line and provide your role ARN
# The role should have necessary permissions for SageMaker operations AmazonSageMakerFullAccess and AmazonS3FullAccess
# role = "arn:aws:iam::123456789012:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001"
region = sagemaker_session.boto_region_name
bucket = sagemaker_session.default_bucket()
print(f"SageMaker role: {role}")
print(f"Region: {region}")
print(f"Bucket: {bucket}")

## Step 2: Configure Data Paths

In [ ]:
bucket = 'your-sagemaker-bucket-name'  # Replace with your actual bucket name
data_path = f's3://{bucket}/segmentation_data/'
output_path = f's3://{bucket}/segmentation_data/output'

print(f"Training data: {data_path}")
print(f"Output path: {output_path}")

## Step 3: Define Hyperparameters

In [ ]:
hyperparameters = {
    "model_name": "SegResNet",
    "batch_size": 2,
    "epochs": 2,
    "val_interval": 2,
    "lr": 1e-4
}

print("Hyperparameters:")
for key, value in hyperparameters.items():
    print(f"  {key}: {value}")

## Step 4: Create SageMaker Estimator

In [ ]:
estimator = PyTorch(
    entry_point="train_simple.py",
    source_dir="../code/training",
    role=role,
    instance_count=1,
    instance_type="ml.g5.xlarge",
    framework_version="2.1.0",
    py_version="py310",
    hyperparameters=hyperparameters,
    output_path=output_path,
    base_job_name="medical-seg-simple",
    keep_alive_period_in_seconds=1800,
    environment={"PIP_CACHE_DIR": "/opt/ml/sagemaker/warmpoolcache/pip"},
    sagemaker_session=sagemaker_session,
)
print("✓ Estimator created successfully!")

## Step 5: Start Training

In [ ]:
estimator.fit({"training": data_path}, wait=True, logs="All")

## Step 6: View Training Results

In [ ]:
model_data = estimator.model_data
training_job_name = estimator.latest_training_job.name
print(f"Training job: {training_job_name}")
print(f"Model artifacts: {model_data}")